# 5:

| No. | Source Area | Source Gantry  | Destination Area | Destination Gantry | Direction |
|-----|-------------|----------------|------------------|---------------------|-----------|
| 1   | Neihu       | 01F0155S       | Yuanshan         | 01F0182S            | S         |
| 2   | Neihu       | 01F0155S       | Taipei           | 01F0248S            | S         |
| 3   | Yuanshan    | 01F0182S       | Taipei           | 01F0248S            | S         |
| 4   | Yuanshan    | 01F0213N       | Neihu            | 01F0155N            | N         |
| 5   | Taipei      | 01F0233N       | Neihu            | 01F0155N            | N         |
| 6   | Taipei      | 01F0233N       | Yuanshan         | 01F0213N            | N         |

# 6, 7 and 8:

In [1]:
import os
import zipfile
import pandas as pd
from datetime import datetime
import glob
import rarfile
import subprocess

# ====== Parameters ======
DATA_DIR = './data'               # Directory containing zip files
TEMP_DIR = './temp'               # Temporary directory to extract files
TARGET_DATE = '20231106'          # Specific date for Task 2
AFTER_TIME = '08:55:00'           # Time threshold for Task 2
VALID_HOURS = list(range(6, 22))  # Only process data from 6:00 to 21:59

# Mapping between areas and gantry IDs
AREA_GANTRIES = {
    "Neihu": ["01F0155N", "01F0155S"],
    "Yuanshan": ["01F0182S", "01F0213N"],
    "Taipei": ["01F0233N", "01F0248S"],
    "Sanzhong": ["01F0256N", "01F0264S"],
    "Wugu": ["01F0293N", "01F0293S"],
    "Freeway Bureau": ["01F0339N", "01F0339S"],
    "Linkou North": ["01F0376N", "01F0376S"],
    "Linkou South": ["01F0413N", "01F0413S"]
}
ALL_GANTRIES = set(g for gs in AREA_GANTRIES.values() for g in gs)
GANTRY_TO_AREA = {g: area for area, gs in AREA_GANTRIES.items() for g in gs}

# ====== Utility Functions ======
def unzip_files():
    os.makedirs(TEMP_DIR, exist_ok=True)
    for fname in os.listdir(DATA_DIR):
        if fname.endswith(('.zip', '.rar')):
            zip_path = os.path.join(DATA_DIR, fname)
            date_str = fname.rsplit('.', 1)[0]  # 去掉扩展名
            subdir = os.path.join(TEMP_DIR, date_str)
            if os.path.exists(subdir):
                print(f"Skipped already unzipped: {fname}")
                continue

            print(f"Extracting: {fname}")
            try:
                if fname.endswith('.zip'):
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        zip_ref.extractall(subdir)
                elif fname.endswith('.rar'):
                    # 使用 unar 解压
                    result = subprocess.run(
                        ['unar', '-o', subdir, '-f', zip_path],
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        text=True
                    )
                    if result.returncode != 0:
                        print(f"Failed to extract {fname} with unar:\n{result.stderr}")
            except Exception as e:
                print(f"Failed to extract {fname}: {e}")

def list_all_csv_files_recursively(root_dir):
    return glob.glob(os.path.join(root_dir, "**", "*.csv"), recursive=True)

def get_hour_from_csv(file_path):
    try:
        df = pd.read_csv(file_path, header=None)
        if df.shape[1] >= 7:
            df = df.iloc[:, :7]
        else:
            return None, None
        df.columns = ["VehicleType", "Time_O", "GantryID_O", "Time_D", "GantryID_D", "TripLength", "TripEnd"]
        if df.empty:
            return None, None
        hour = int(df.iloc[0]["Time_O"].split(" ")[1].split(":")[0])
        return df, hour
    except Exception as e:
        print(f"Failed to read: {file_path} - {e}")
        return None, None

def get_duration(start, end):
    try:
        t1 = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
        t2 = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
        return (t2 - t1).total_seconds()
    except:
        return None

def seconds_to_mmss(sec):
    if pd.isna(sec) or sec is None:
        return "--:--"
    minutes = int(sec // 60)
    seconds = int(sec % 60)
    return f"{str(minutes).zfill(2)}:{seconds:02d}"

# ====== Task 2 ======
def task2():
    print("\nTask 2: First 10 vehicles from Neihu to Taipei after 08:55 on 2023-11-06")
    target_path = os.path.join(TEMP_DIR, TARGET_DATE)
    if not os.path.exists(target_path):
        print("Target date directory does not exist. Make sure the zip has been extracted.")
        return

    files = list_all_csv_files_recursively(target_path)
    if not files:
        print("No CSV files found.")
        return

    all_rows = []
    for f in sorted(files):
        print(f"\nReading file: {os.path.basename(f)}")
        df, _ = get_hour_from_csv(f)
        if df is None:
            print("Unable to read or format error")
            continue

        print(f"Total records: {len(df)}")
        df1 = df[df["TripEnd"] == "Y"]
        print(f"1. TripEnd=Y: {len(df1)}")

        df2 = df1[df1["GantryID_O"].isin(AREA_GANTRIES["Neihu"])]
        print(f"2. From Neihu: {len(df2)}")

        df3 = df2[df2["GantryID_D"].isin(AREA_GANTRIES["Taipei"])]
        print(f"3. To Taipei: {len(df3)}")

        df4 = df3[df3["GantryID_O"].str[-1] == df3["GantryID_D"].str[-1]]
        print(f"4. Same direction: {len(df4)}")

        df4["Time_O_dt"] = pd.to_datetime(df4["Time_O"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
        df5 = df4[df4["Time_O_dt"].dt.time > pd.to_datetime(AFTER_TIME).time()].copy()
        print(f"5. Entry time > 08:55: {len(df5)}")

        df5["Duration"] = df5.apply(lambda row: get_duration(row["Time_O"], row["Time_D"]), axis=1)
        df5 = df5.dropna(subset=["Duration"])
        print(f"6. Valid durations: {len(df5)}")

        all_rows.extend(df5[["Time_O", "Time_D", "Duration"]].values.tolist())
        if len(all_rows) >= 10:
            break

    if all_rows:
        df_all = pd.DataFrame(all_rows, columns=["Time_O", "Time_D", "Duration"])
        df_all["Time_O_dt"] = pd.to_datetime(df_all["Time_O"])
        df_all = df_all.sort_values("Time_O_dt").head(10)

        print(f"\nFinal {len(df_all)} records:")
        print(f"{'Entry Time':>19}  {'Exit Time':>19}  {'Duration'}")
        for _, row in df_all.iterrows():
            print(f"{row['Time_O']:>19}  {row['Time_D']:>19}  {seconds_to_mmss(row['Duration'])}")
    else:
        print("No valid records found.")

# ====== Task 3 & 4 ======
def task3_and_4():
    print("\nTask 3 & 4: Construct 8x8 travel time table and traffic flow matrix")
    travel_data = []
    valid_hour_count = 0
    all_csv_files = list_all_csv_files_recursively(TEMP_DIR)
    print(f"Found {len(all_csv_files)} csv files")

    for f in all_csv_files:
        df, hour = get_hour_from_csv(f)
        if df is None or hour not in VALID_HOURS:
            continue

        df = df[(df["TripEnd"] == "Y") &
                (df["GantryID_O"].isin(ALL_GANTRIES)) &
                (df["GantryID_D"].isin(ALL_GANTRIES)) &
                (df["GantryID_O"].str[-1] == df["GantryID_D"].str[-1]) &
                (df["GantryID_O"] != df["GantryID_D"])
                ]
        if df.empty:
            continue

        df["Duration"] = df.apply(lambda row: get_duration(row["Time_O"], row["Time_D"]), axis=1)
        df = df.dropna(subset=["Duration"])
        df["From"] = df["GantryID_O"].map(GANTRY_TO_AREA)
        df["To"] = df["GantryID_D"].map(GANTRY_TO_AREA)
        travel_data.append(df[["From", "To", "Duration"]])
        valid_hour_count += 1  # 有效文件代表一个小时段

    if not travel_data:
        print("No valid data.")
        return

    full = pd.concat(travel_data, ignore_index=True)
    area_order = list(AREA_GANTRIES.keys())

    # === 平均通行时间（秒） ===
    pivot_time_sec = full.groupby(["From", "To"])["Duration"].mean().unstack()
    pivot_time_sec = pivot_time_sec.reindex(index=area_order, columns=area_order).fillna("-")

    # === 平均通行时间（分钟，保留 2 位小数） ===
    pivot_time_min = pivot_time_sec.copy()
    pivot_time_min = pivot_time_min.applymap(lambda x: round(x / 60, 2) if isinstance(x, (int, float)) else x)
    
    def format_time_table(df):
        return df.astype(object).apply(lambda col: col.map(lambda x: seconds_to_mmss(x) if isinstance(x, (int, float)) else x))
    pivot_time_mmss = format_time_table(pivot_time_sec)

    total_flow = full.groupby(["From", "To"]).size()
    
    print(f"\nValid hourly files counted: {valid_hour_count}")
    hour_coverage = valid_hour_count
    
    # Step 4: 平均每小时车辆数
    avg_flow = total_flow / hour_coverage

    # Step 5: 构建 8x8 表格
    pivot_flow = avg_flow.unstack()
    pivot_flow = pivot_flow.reindex(index=area_order, columns=area_order).fillna("-")
    
    print("\nAverage journey time table (in minutes):")
    print(pivot_time_min)
    pivot_time_min.to_csv("average_journey_time_minutes.csv")

    print("\nAverage journey time table (MM:SS):")
    print(pivot_time_mmss)
    pivot_time_mmss.to_csv("average_journey_time_mmss.csv")

    print("\nAverage traffic flow table (vehicles per hour):")
    print(pivot_flow)
    pivot_flow.to_csv("average_flow_matrix.csv")


In [2]:
if __name__ == "__main__":
    unzip_files()
    task2()
    task3_and_4()

Skipped already unzipped: 20231127.zip
Skipped already unzipped: 20231130.zip
Skipped already unzipped: 20231124.zip
Skipped already unzipped: 20231120.zip
Skipped already unzipped: 20231108.zip
Skipped already unzipped: 20231109.zip
Skipped already unzipped: 20231121.zip
Skipped already unzipped: 20231123.zip
Skipped already unzipped: 20231122.zip
Skipped already unzipped: 20231107.rar
Skipped already unzipped: 20231113.zip
Skipped already unzipped: 20231106.zip
Skipped already unzipped: 20231110.zip
Skipped already unzipped: 20231115.zip
Skipped already unzipped: 20231129.zip
Skipped already unzipped: 20231128.zip
Skipped already unzipped: 20231114.zip
Skipped already unzipped: 20231116.zip
Skipped already unzipped: 20231117.zip
Skipped already unzipped: 20231206.zip
Skipped already unzipped: 20231205.zip
Skipped already unzipped: 20231204.zip
Skipped already unzipped: 20231201.zip

Task 2: First 10 vehicles from Neihu to Taipei after 08:55 on 2023-11-06

Reading file: 23110606.csv
T

/var/folders/vt/qwvfy06n63q184jd56rxd3dh0000gn/T/ipykernel_79636/699532114.py:194: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_time_min = pivot_time_min.applymap(lambda x: round(x / 60, 2) if isinstance(x, (int, float)) else x)



Valid hourly files counted: 368

Average journey time table (in minutes):
To               Neihu Yuanshan  Taipei Sanzhong   Wugu Freeway Bureau  \
From                                                                     
Neihu                -     3.19   11.93    13.98  15.11          21.35   
Yuanshan          7.39        -    8.26    10.18  11.91          18.08   
Taipei            9.52     1.85       -     1.33    3.3           8.04   
Sanzhong         12.69     4.93    3.13        -   2.21           8.02   
Wugu             17.31     9.87    7.46     4.11      -            5.1   
Freeway Bureau  169.33        -  177.62        -      -              -   
Linkou North      25.8    18.61   16.25    12.97    8.3              -   
Linkou South     45.43    31.94    26.9    22.29  12.64              -   

To             Linkou North Linkou South  
From                                      
Neihu                 22.24        30.01  
Yuanshan              19.12        25.02  
Taipei      

## Task 2: First 10 vehicles from Neihu to Taipei after 08:55 on 2023-11-06

| Entry Time          | Exit Time           | Duration |
|---------------------|---------------------|----------|
| 2023-11-06 08:55:12 | 2023-11-06 09:02:59 | 07:47    |
| 2023-11-06 08:55:30 | 2023-11-06 09:04:07 | 08:37    |
| 2023-11-06 08:55:46 | 2023-11-06 09:04:32 | 08:46    |
| 2023-11-06 08:56:13 | 2023-11-06 09:04:43 | 08:30    |
| 2023-11-06 08:56:31 | 2023-11-06 09:05:09 | 08:38    |
| 2023-11-06 08:56:32 | 2023-11-06 09:04:59 | 08:27    |
| 2023-11-06 08:56:35 | 2023-11-06 09:04:56 | 08:21    |
| 2023-11-06 08:56:39 | 2023-11-06 09:05:24 | 08:45    |
| 2023-11-06 08:56:52 | 2023-11-06 09:05:29 | 08:37    |
| 2023-11-06 08:56:53 | 2023-11-06 09:05:21 | 08:28    |

## Average journey time table (in minutes):

| From          | Neihu | Yuanshan | Taipei | Sanzhong | Wugu  | Freeway Bureau | Linkou North | Linkou South |
|---------------|--------|----------|--------|-----------|--------|-----------------|---------------|----------------|
| Neihu         | -      | 3.19     | 11.93  | 13.98     | 15.11 | 21.35          | 22.24        | 30.01         |
| Yuanshan      | 7.39   | -        | 8.26   | 10.18     | 11.91 | 18.08          | 19.12        | 25.02         |
| Taipei        | 9.52   | 1.85     | -      | 1.33      | 3.30  | 8.04           | 9.97         | 13.83         |
| Sanzhong      | 12.69  | 4.93     | 3.13   | -         | 2.21  | 8.02           | 9.24         | 14.12         |
| Wugu          | 17.31  | 9.87     | 7.46   | 4.11      | -     | 5.10           | 6.82         | 11.63         |
| Freeway Bureau| 169.33 | -        | 177.62 | -         | -     | -              | 3.01         | 8.38          |
| Linkou North  | 25.80  | 18.61    | 16.25  | 12.97     | 8.30  | -              | -            | 8.62          |
| Linkou South  | 45.43  | 31.94    | 26.90  | 22.29     | 12.64 | -              | 6.07         | -             |

## Average journey time table (MM:SS):

| From          | Neihu | Yuanshan | Taipei | Sanzhong | Wugu  | Freeway Bureau | Linkou North | Linkou South |
|---------------|--------|----------|--------|-----------|--------|-----------------|---------------|----------------|
| Neihu         | -      | 03:11    | 11:55  | 13:58     | 15:06 | 21:20          | 22:14        | 30:00         |
| Yuanshan      | 07:23  | -        | 08:15  | 10:10     | 11:54 | 18:05          | 19:06        | 25:01         |
| Taipei        | 09:31  | 01:50    | -      | 01:19     | 03:18 | 08:02          | 09:58        | 13:50         |
| Sanzhong      | 12:41  | 04:55    | 03:07  | -         | 02:12 | 08:00          | 09:14        | 14:07         |
| Wugu          | 17:18  | 09:51    | 07:27  | 04:06     | -     | 05:05          | 06:48        | 11:37         |
| Freeway Bureau| 169:20 | -        | 177:37 | -         | -     | -              | 03:00        | 08:22         |
| Linkou North  | 25:48  | 18:36    | 16:15  | 12:58     | 08:17 | -              | -            | 08:37         |
| Linkou South  | 45:26  | 31:56    | 26:54  | 22:17     | 12:38 | -              | 06:04        | -             |

## Average traffic flow table (vehicles per hour):

| From          | Neihu     | Yuanshan  | Taipei    | Sanzhong  | Wugu     | Freeway Bureau | Linkou North | Linkou South |
|---------------|-----------|-----------|-----------|-----------|----------|----------------|---------------|----------------|
| Neihu         | -         | 131.652   | 193.693   | 73.508    | 99.106   | 0.644          | 59.435        | 0.016         |
| Yuanshan      | 132.016   | -         | 310.595   | 141.726   | 147.136  | 2.198          | 84.535        | 0.021         |
| Taipei        | 129.421   | 287.611   | -         | 359.530   | 419.402  | 6.899          | 532.334       | 0.128         |
| Sanzhong      | 65.829    | 170.747   | 531.410   | -         | 256.016  | 2.516          | 218.264       | 0.052         |
| Wugu          | 57.799    | 122.848   | 467.136   | 190.253   | -        | 2.299          | 160.957       | 0.054         |
| Freeway Bureau| 0.005     | -         | 0.003     | -         | -        | -              | 899.149       | 0.277         |
| Linkou North  | 37.845    | 86.932    | 556.076   | 223.984   | 151.079  | -              | -             | 0.209         |
| Linkou South  | 0.008     | 0.038     | 0.149     | 0.082     | 0.071    | -              | 0.234         | -             |